In [1]:
# Native
import re
import sys
import time

# Third party
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

# Custom
sys.path.append("..")
from linkedInCred import (linkedIn_email, linkedIn_password)
from scripts.companyObjects import (daugherty, slalom, _1904labs, worldWideTechnology)

In [2]:
# from Classes.DB import DB
# web_data_db = DB()
# web_data_db.connect()
# web_data_db.createTableCompanyProfiles()
# web_data_db.createTableCompanyPosts()

# web_data_db.intoCompanyProfiles(company.name, followers, employees_on_linkedin, url)
# web_data_db.select()
# web_data_db.intoCompanyPosts(content, like_count, comment_count, date, id + 1)
# web_data_db.select()

In [3]:
companies = [slalom, _1904labs, worldWideTechnology]

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.driver.set_window_size(1600, 900) # Has to be wide enough to prevent messenger from covering filter buttons

In [5]:
def logOnToLinkedIn():
    browser.visit(companies[0].linkedin)
    button = browser.links.find_by_partial_href("https://www.linkedin.com/login?")
    button.click()
    browser.fill('session_key', linkedIn_email)
    browser.fill('session_password', linkedIn_password)
    button = browser.find_by_value('Sign in')
    button.click()

In [6]:
def scrapeProfileMetaData(company):
    url = company.linkedin
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    data = {}
    
    followers = re.search(r'[\d,]+ followers', html, re.DOTALL).group()
    followers = re.sub("[^\d]", "", followers) # returns only digits
    followers = int(followers)
    
    employees_on_linkedin = re.search(r'[\d,]+ employees', html, re.DOTALL).group()
    employees_on_linkedin = re.sub("[^\d]", "", employees_on_linkedin) # returns only digits
    employees_on_linkedin = int(employees_on_linkedin)

    print(f"{company.name} has {followers} followers and {employees_on_linkedin} employees on LinkedIn.")
    
    data["name"] = [company.name]
    data["followers"] = [followers]
    data["employees_on_linkedin"] = [employees_on_linkedin]
    
    profile_metadata_df = pd.DataFrame(data=data)
    profile_metadata_df.to_csv(f"../data/{company.name}_profile_metadata")

In [7]:
def sortProfilePostsByMostRecent():
    button = browser.find_by_css("div[class='sort-dropdown mt2 ember-view']")
    button.click()
    button = browser.find_by_text('Recent')
    button.click()

In [8]:
def scrollDownUntilAllPostsAreLoaded():
    number_posts_before_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
    
    while number_posts_before_scroll > 1:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        number_posts_after_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
        
        if (number_posts_before_scroll == number_posts_after_scroll):
            timer = time.time()
            thirty_seconds_elapsed = timer + 30
            
            while time.time() < thirty_seconds_elapsed:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                number_posts_after_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
                
            if (number_posts_before_scroll == number_posts_after_scroll):
                print("finished")
                break
        else:
            number_posts_before_scroll = len(browser.find_by_css("div[class='occludable-update ember-view']"))
                
    print(f"Number of posts {number_posts_before_scroll}")   

In [9]:
def scrapeCompanyPosts(id):
    posts = browser.find_by_css("div[class='occludable-update ember-view']")
    
    data = {
        "content": []
        , "like_count": []
        , "comment_count": []
        , "date": []
    }
    
    for post in posts:
        post = post.text
          
        try:
            date = re.search(r'^(\w+ •\n|\w+ ago\n)', post, re.MULTILINE).group()
            date = re.sub("\n", "", date)
        except Exception as e:
            print(f"no date: {e}")
            date = ""
            
        try:
            like_count = re.search(r'^[\d]+$', post, re.MULTILINE).group()
            like_count = int(like_count)
        except Exception as e:
            print(f"no like count: {e}")
            like_count = 0

        try:
            # Content always follows the time the post was published and precedes the like count.
            contentRegex = re.compile(f"( •\n| ago\n).*?^({like_count})$", re.MULTILINE|re.DOTALL)
            content = re.search(contentRegex, post).group()
            content = re.sub(r"( •\n| ago\n)", "", content, re.MULTILINE|re.DOTALL) # Gets rid of the leading timestamp
            content = content[:-len(str(like_count))] # Gets rid of the trailing like count
        except Exception as e:
            print(f"no content: {e}")
            content = ""
            
        try:
            comment_area = re.search(r'^[\d]+ comment(s)?$', post, re.MULTILINE).group()
            comment_count = re.sub(r"[^\d]", "", comment_area) # returns only digits
            comment_count = int(comment_count)
        except Exception as e:
            print(f"no comment count: {e}")
            comment_count = 0
            
        print(f"\n\nlikes: {like_count}")
        print(f"comments: {comment_count}")
        print(f"content: {content}")
        
        data["content"].append(content)
        data["like_count"].append(like_count)
        data["comment_count"].append(comment_count)
        data["date"].append(date)
        
    company_posts_df = pd.DataFrame(data)
    company_posts_df.to_csv(f"../data/{companies[id].name}_company_posts")   

In [10]:
logOnToLinkedIn()

for id in range(len(companies)):
    scrapeProfileMetaData(companies[id])
    browser.visit(companies[id].linkedin)
    sortProfilePostsByMostRecent()
    scrollDownUntilAllPostsAreLoaded()
    scrapeCompanyPosts(id)

Slalom has 116261 followers and 7511 employees on LinkedIn.
finished
Number of posts 1135
no comment count: 'NoneType' object has no attribute 'group'


likes: 8
comments: 0
content: There are thousands of amazing women at Slalom, and after our recognition as one of the #BestWorkplaces for women, we gathered some of their perspectives on work, life, leadership, and transformation: https://slalom.ws/2W9zFD4 #CelebrateAuthenticity
…see more
Slalom women: Words of wisdom
slalom.com • 1 min read

no comment count: 'NoneType' object has no attribute 'group'


likes: 3
comments: 0
content: #AI is changing virtually every industry, and can transform your customer experience. Learn how here: https://bit.ly/3mu6DJm
Download the whitepaper
go.slalom.com • 1 min read

no comment count: 'NoneType' object has no attribute 'group'


likes: 3
comments: 0
content: With a modernized loan processing system in place that can easily handle new products and process billions of dollars daily, Federal Home L

no comment count: 'NoneType' object has no attribute 'group'


likes: 10
comments: 0
content: Join Slalom and Salesforce experts to learn how #datadriven insights can uncover new business opportunities across your ecosystem: https://bit.ly/35HaLQY
Register for the Future of Gaming webinar
event.on24.com • 1 min read

no date: 'NoneType' object has no attribute 'group'
no comment count: 'NoneType' object has no attribute 'group'


likes: 12
comments: 0
content: Only 11% of healthcare consumers strongly agree that communications from their providers feel relevant and personal to them. How do we create a better patient experience? Download our whitepaper with Slalom and Salesforce experts to learn more: https://slalom.ws/2Vw29Xx
…see more
Download the whitepaper
go.slalom.com • 1 min read

no comment count: 'NoneType' object has no attribute 'group'


likes: 12
comments: 0
content: Creating an innovative #digitalproduct is complex, especially in #lifesciences where speed, accuracy, and sa

no comment count: 'NoneType' object has no attribute 'group'


likes: 17
comments: 0
content: Coaching through a Crisis, our second program in the Slalom Presents virtual series, explored how great coaching can change the game. To continue building your coaching habits, we put together insights and resources to help you build strong, resilient teams. Learn more here: https://slalom.ws/35Fdt9y
…see more
Activate link to view larger image.

no comment count: 'NoneType' object has no attribute 'group'


likes: 94
comments: 0
content: Congratulations to Alex Hatcher, a solution architect in Slalom New York City, for achieving his 6th specialty Amazon Web Services (AWS) certification—making this his 12th certification overall! Only a handful of people at Slalom have achieved all 12 AWS certifications which earns them the distinction of Cloudius Maximus.
…see more
Activate link to view larger image.

no comment count: 'NoneType' object has no attribute 'group'


likes: 48
comments: 0
content

no comment count: 'NoneType' object has no attribute 'group'


likes: 10
comments: 0
content: Join Slalom for a game-changing look at coaching and leadership. We’ll hear from legendary coaches like Jill Ellis, former coach of the US Women’s National Soccer Team who led the team to two consecutive World Cup wins in 2015 and 2019. Learn how you can build your own coaching habits to fuel connection and growth, even in extraordinary times:
…see more
Join us November 18 for Slalom Presents
go.slalom.com • 1 min read



likes: 247
comments: 7
content: Excited to announce that we're #3 on the list! Thank you to all our employees for making Slalom one of the best firms to work for in 2020. See the other winners here: https://bit.ly/32Clydm
Loaded:
Play
Back to start of video
Current time 
0:00
/
Duration 
0:11
 Unmute
Turn fullscreen on
Slalom recognized as a 2020 Best Firm to Work For

no comment count: 'NoneType' object has no attribute 'group'


likes: 9
comments: 0
content: On a scale from

no comment count: 'NoneType' object has no attribute 'group'


likes: 25
comments: 0
content: “We can create long-lasting change with the right intent,” says Aishwarya. "I believe in people to my core.”

We’re grateful for incredible contributors like Aishwarya Bhadouria who make Slalom a Best Workplace for Women. Learn more about what motivates the Forbes 30 under 30 honoree, black belt martial artist, and engineering superstar here:
…see more
Activate link to view larger image.

no date: 'NoneType' object has no attribute 'group'


likes: 100
comments: 2
content: Extraordinary times call for extraordinary coaches. Hosted by the award-winning ESPN anchor Hannah Storm, we’re honored to have Earvin "Magic" Johnson, former Coach Jill Ellis, Wing Commander Andy Green, and Arshay Cooper as speakers for our next Slalom Presents: Coaching through a Crisis.

These legendary coaches have led teams through challenges that extend beyond the game—whether they’re setting records or pushing the lim

...

no comment count: 'NoneType' object has no attribute 'group'


likes: 8
comments: 0
content: How do companies reach a point where their data becomes completely chaotic and cumbersome? Find out how Data Discovery & Analysis can make your data manageable again in our latest blog post: https://lnkd.in/efwhTNB #datadiscovery #data #analysis #dataengineering #team1904
…see more
How Data Discovery and Analysis Can Make Your Data Manageable Again - 1904labs
1904labs.com

no comment count: 'NoneType' object has no attribute 'group'


likes: 6
comments: 0
content: Ever wonder what it's like to go back to your alma mater and mentor 300 high school and college students from around the country over 36 hrs for a hackathon? Brandon Belvin did just that recently for PickHacks, all the details here:
https://lnkd.in/etNeQGy #team1904 #hackathon #mentor #community
…see more
Seeing the Startup Mindset in Action at PickHacks 2019
1904labs.com

no comment count: 'NoneType' object has no attribute 'group'



no comment count: 'NoneType' object has no attribute 'group'


likes: 16
comments: 0
content: Thanks for the shoutout Prepare.ai. We are really excited to be a first time sponsor at this years conference on April 9th! Who else is planning to attend? #ai #decisionscience #Technology #team1904
Prepare.ai
479 followers
1yr1 yearPrepare 2019 wouldn’t be possible without the generous support of 1904Labs!

1904labs helps some of the world’s largest companies innovate, bringing them the nimbleness of a startup without the risks, challenges, or costs associated with building new teams internally.

Thank you 1904labs!



Grab the few last remaining tickets for the 2019 Prepare conference here:
…see more
Activate link to view larger image.



likes: 8
comments: 1
content: Happy International Woman's Day from the women of 1904labs! Our newest blog, How Companies Benefit from More Women in Technology, is a must read, especially today! https://lnkd.in/e5mpFAs #WIT #womenintech #team1904 #Internatio

no comment count: 'NoneType' object has no attribute 'group'


likes: 8
comments: 0
content: Here at 1904labs we are constantly evolving & growing. Check out some recent updates we made to our website to reflect who we are as a company. #1904labs #Technology #StartUpMinds #InnovationSolutions #BuildingTheRightThing
Home - 1904labs
1904labs.com

no comment count: 'NoneType' object has no attribute 'group'


likes: 12
comments: 0
content: Today we honor all the men and women that have served and are currently serving our country.

We also proudly recognize our own Maxwell Claypool, 1904labs Full Stack Developer, for his service in the United States Navy,, Full Stack Developer, for his service in the United States Air Force Reserve and, Director of Business Development, for his service in the United States National Guard. We are so grateful to each of you!
…see more
Activate link to view larger image.

no comment count: 'NoneType' object has no attribute 'group'


likes: 9
comments: 0
con

no comment count: 'NoneType' object has no attribute 'group'


likes: 7
comments: 0
content: All gave some, some gave all, today we Remember, Celebrate and Honor each of them. Thank you to the men and women that have served or are currently serving our country! Happy Memorial Day from 1904labs #grateful #team1904
Activate link to view larger image.



likes: 19
comments: 1
content: Introducing our Spring Class of 2018! When you come work with @1904labs, we get all the new hires together for a welcome lunch...who says there is no free lunch? :-)

We are so excited to have each and every one of you as part of #team1904! #classof2018           
…see more
Activate link to view larger image.

no comment count: 'NoneType' object has no attribute 'group'


likes: 2
comments: 0
content: 1904labs would be interested in your thoughts on HCDAgile by having you take a short survey. As a reward for your time, one respondent will be able to direct $1,000 to the charity of their choice! Take the surv

no comment count: 'NoneType' object has no attribute 'group'


likes: 3
comments: 0
content: Attending #GeoYouSTL? Make sure you catch up withJohn Ross, 1904labs GIS Analyst, who is a lead in a Startup/Entrepreneurship brainstorming session today, then tomorrow he is giving a flash talk on open source GIS. www.geoyou.org
Welcome
geoyou.org

no like count: 'NoneType' object has no attribute 'group'
no content: 'NoneType' object has no attribute 'group'
no comment count: 'NoneType' object has no attribute 'group'


likes: 0
comments: 0
content: 


likes: 17
comments: 2
content: 1904labs was part of the biggest office party in St. Louis tonight at the Biz Dash 5K. Congratulations to everyone and especially Sean Walsh who took top 5 in the CEO Challenge!
Activate link to view larger image.

no comment count: 'NoneType' object has no attribute 'group'


likes: 16
comments: 0
content: The new 1904labs shirts arrived and appear to be a hit with the team...you guys wear it well! #corebeliefs
A